<a href="https://colab.research.google.com/github/punam-gwachha/CNN/blob/main/FTasKIII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation

In [11]:
# Step 1: Install required libraries
!pip install nibabel torch torchvision scikit-learn pandas

In [12]:
# Step 2: Import modules
import nibabel as nib
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import os

In [13]:
# from google.colab import files
# Step 3: Upload files
print("Please upload all CT and mask .nii.gz files")
uploaded = files.upload()

Please upload all CT and mask .nii.gz files


Saving 3702_left_knee.nii.gz to 3702_left_knee.nii.gz
Saving segmented_femur_tibia.nii.gz to segmented_femur_tibia.nii.gz


In [18]:
# Step 4: Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
print("Using device:", device)

Using device: cpu


##Convert 2D Pretrained Model to 3D:

In [14]:

#Step 5: Inflation Helpers

def to_2tuple(x):
    return (x, x) if isinstance(x, int) else x

def inflate_conv(conv2d, depth=3):
    conv3d = nn.Conv3d(
        conv2d.in_channels, conv2d.out_channels,
        kernel_size=(depth, *to_2tuple(conv2d.kernel_size)),
        stride=(1, *to_2tuple(conv2d.stride)),
        padding=(depth // 2, *to_2tuple(conv2d.padding)),
        dilation=(1, *to_2tuple(conv2d.dilation)),
        bias=conv2d.bias is not None
    )
    with torch.no_grad():
        w2d = conv2d.weight.data
        w3d = w2d.unsqueeze(2).repeat(1, 1, depth, 1, 1) / depth
        conv3d.weight.copy_(w3d)
        if conv2d.bias is not None:
            conv3d.bias.copy_(conv2d.bias.data)
    return conv3d

def inflate_batchnorm(bn2d):
    return nn.BatchNorm3d(
        num_features=bn2d.num_features,
        eps=bn2d.eps,
        momentum=bn2d.momentum,
        affine=bn2d.affine,
        track_running_stats=bn2d.track_running_stats
    )

def inflate_pool(pool2d):
    if isinstance(pool2d, nn.AdaptiveAvgPool2d):
        return nn.AdaptiveAvgPool3d((1, 1, 1))
    elif isinstance(pool2d, nn.AvgPool2d):
        return nn.AvgPool3d(
            kernel_size=(1, *to_2tuple(pool2d.kernel_size)),
            stride=(1, *to_2tuple(pool2d.stride)),
            padding=(0, *to_2tuple(pool2d.padding))
        )
    elif isinstance(pool2d, nn.MaxPool2d):
        return nn.MaxPool3d(
            kernel_size=(1, *to_2tuple(pool2d.kernel_size)),
            stride=(1, *to_2tuple(pool2d.stride)),
            padding=(0, *to_2tuple(pool2d.padding)),
            dilation=(1, *to_2tuple(pool2d.dilation)),
            ceil_mode=pool2d.ceil_mode
        )
    return pool2d

def inflate_densenet(model_2d):
    for name, module in model_2d.named_children():
        if isinstance(module, nn.Conv2d):
            setattr(model_2d, name, inflate_conv(module))
        elif isinstance(module, nn.BatchNorm2d):
            setattr(model_2d, name, inflate_batchnorm(module))
        elif isinstance(module, (nn.MaxPool2d, nn.AvgPool2d, nn.AdaptiveAvgPool2d)):
            setattr(model_2d, name, inflate_pool(module))
        elif isinstance(module, nn.Sequential):
            for subname, submodule in module.named_children():
                if isinstance(submodule, nn.Conv2d):
                    setattr(module, subname, inflate_conv(submodule))
                elif isinstance(submodule, nn.BatchNorm2d):
                    setattr(module, subname, inflate_batchnorm(submodule))
                elif isinstance(submodule, (nn.MaxPool2d, nn.AvgPool2d, nn.AdaptiveAvgPool2d)):
                    setattr(module, subname, inflate_pool(submodule))
                else:
                    inflate_densenet(submodule)
        else:
            inflate_densenet(module)
    return model_2d

#Feature Extraction

In [15]:
# Step 6: Feature Hooks and Extractors
feature_maps = {}

def get_activation(name):
    def hook(model, input, output):
        feature_maps[name] = output
    return hook

def register_hooks(model):
    conv_layers = [m for m in model.modules() if isinstance(m, nn.Conv3d)]
    for name, layer in zip(['last', 'third_last', 'fifth_last'], [conv_layers[-1], conv_layers[-3], conv_layers[-5]]):
        layer.register_forward_hook(get_activation(name))

def global_avg_pool(tensor):
    return F.adaptive_avg_pool3d(tensor, 1).squeeze()

def extract_features_chunked(model, input_tensor, chunk_depth=16):
    feature_maps.clear()
    b, c, d, h, w = input_tensor.shape
    intermediate = {'last': [], 'third_last': [], 'fifth_last': []}
    model.eval()
    with torch.no_grad():
        for start in range(0, d, chunk_depth):
            end = min(start + chunk_depth, d)
            chunk = input_tensor[:, :, start:end, :, :]
            _ = model(chunk)
            for key in intermediate:
                if key in feature_maps:
                    pooled = global_avg_pool(feature_maps[key])
                    intermediate[key].append(pooled.detach().cpu())
    return {k: torch.stack(v).mean(0).numpy() for k, v in intermediate.items() if v}

def compute_cosine_similarity(feat1, feat2):
    return {k: cosine_similarity([feat1[k]], [feat2[k]])[0][0] for k in feat1}

def prepare(volume):
    tensor = torch.tensor(volume).float().unsqueeze(0).unsqueeze(0)
    return tensor.repeat(1, 3, 1, 1, 1).to(device)

In [16]:
# Step 7: Load and Inflate Model
model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
model = inflate_densenet(model)
model.classifier = nn.Identity()  # Remove final linear layer
model = model.to(device)
register_hooks(model)
model.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (norm0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=(1, 1, 1), ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv3d(64, 128, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
        (norm2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv3d(128, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm3d(96, ep

In [9]:
#feature comparision
# Step 8: Manual Single File Mode (optional override)
ct_path = '/content/3702_left_knee.nii.gz'
mask_path = '/content/segmented_femur_tibia.nii.gz'

if os.path.exists(ct_path) and os.path.exists(mask_path):
    print(f"Manually processing: {ct_path} and {mask_path}")
    ct = nib.load(ct_path).get_fdata().astype(np.float32)
    mask_raw = nib.load(mask_path).get_fdata()

    # Reduce dimensions on mask to match CT shape
    mask = mask_raw
    while mask.ndim > 3:
        mask = mask[..., 0]
    mask = mask.astype(np.uint8)

    if ct.shape != mask.shape:
        print(f"Shape mismatch on manual file: CT {ct.shape}, Mask {mask.shape}")
    else:
        # Normalize CT to [0, 1] and threshold small negatives to 0
        ct = np.clip(ct, a_min=0, a_max=None)
        if np.max(ct) > 0:
            ct = ct / np.max(ct)

        tibia = (mask == 1) * ct
        femur = (mask == 2) * ct
        background = (mask == 0) * ct

        # # Skip empty masks
        # if np.count_nonzero(tibia) < 100 or np.count_nonzero(femur) < 100:
        #     print("Tibia or femur region too small. Skipping manual file.")
        # else:
        tibia_tensor = prepare(tibia)
        femur_tensor = prepare(femur)
        background_tensor = prepare(background)

        f_tibia = extract_features_chunked(model, tibia_tensor)
        f_femur = extract_features_chunked(model, femur_tensor)
        f_bg = extract_features_chunked(model, background_tensor)

        sim_tf = compute_cosine_similarity(f_tibia, f_femur)
        sim_tb = compute_cosine_similarity(f_tibia, f_bg)
        sim_fb = compute_cosine_similarity(f_femur, f_bg)

        results = [{
            'image_id': '3702_left_knee',
            'Tibia-Femur (last)': sim_tf['last'],
            'Tibia-Femur (third_last)': sim_tf['third_last'],
            'Tibia-Femur (fifth_last)': sim_tf['fifth_last'],
            'Tibia-Background (last)': sim_tb['last'],
            'Tibia-Background (third_last)': sim_tb['third_last'],
            'Tibia-Background (fifth_last)': sim_tb['fifth_last'],
            'Femur-Background (last)': sim_fb['last'],
            'Femur-Background (third_last)': sim_fb['third_last'],
            'Femur-Background (fifth_last)': sim_fb['fifth_last'],
        }]

Manually processing: /content/3702_left_knee.nii.gz and /content/segmented_femur_tibia.nii.gz


In [17]:
# Step 9: Save Results
output_df = pd.DataFrame(results)
output_df.to_csv("feature_similarity_batch.csv", index=False)
print("All features saved to feature_similarity_batch.csv")

All features saved to feature_similarity_batch.csv
